# Experiment: Random walk CPM 
The goal of this experiment is to apply gradient-based learning to a simplified CPM model simulating a random walk 

import modules

In [1]:
import torch as t
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../../")
from periodic_padding import periodic_padding
from hamiltonian_diff import model
from cell_typing import CellKind, CellMap

## 1. Splitting the grid into subdomains for parallel checkerboarding update scheme

In [2]:
def split_grids(batch:t.Tensor, kernel_width: int, kernel_height: int) -> t.Tensor:
    unfold_transform = t.nn.Unfold(kernel_size=(kernel_height,kernel_width))
    
    padded_batch = periodic_padding(batch).float()
    padded_batch = padded_batch.unsqueeze(1)
    
    return t.transpose(unfold_transform(padded_batch), dim0=1, dim1=2)

The goal is to split the grid into smaller subdomains of a given size. E.g. we may want to create subgrids of size 2x2. To update these subdomains, we also need the Moore Neighborhood around them. 

To achieve that, we can apply periodic padding and then use an Unfolding convolution layer. Using a kernel size of $k=(4,4)$ (subdomain dims + 2) we get convolutional blocks containing the desired 2x2 subgrids + Moore neighborhood, unrolled into vectors (one vector for each convolutional block).

In [3]:
test = t.arange(2*4*4).reshape(2,4,4)
print(test)

split = split_grids(test, kernel_width=4, kernel_height=4)
print(split.shape)

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15]],

        [[16, 17, 18, 19],
         [20, 21, 22, 23],
         [24, 25, 26, 27],
         [28, 29, 30, 31]]])
torch.Size([2, 9, 16])


we can also use different kernel sizes for the width and height. E.g. we may want to only consider 1x2 subgrids:

In [4]:
test = t.arange(2*4*4).reshape(2,4,4)
print(test)

split = split_grids(test, kernel_width=4, kernel_height=3)
print(split.shape)

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15]],

        [[16, 17, 18, 19],
         [20, 21, 22, 23],
         [24, 25, 26, 27],
         [28, 29, 30, 31]]])
torch.Size([2, 12, 12])


To apply checkerboarding, we want to choose a set of subgrids where there is at least 2 columns/rows between them:

In [5]:
test = t.arange(64.).reshape(1,8,8)
print(test)

tensor([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11., 12., 13., 14., 15.],
         [16., 17., 18., 19., 20., 21., 22., 23.],
         [24., 25., 26., 27., 28., 29., 30., 31.],
         [32., 33., 34., 35., 36., 37., 38., 39.],
         [40., 41., 42., 43., 44., 45., 46., 47.],
         [48., 49., 50., 51., 52., 53., 54., 55.],
         [56., 57., 58., 59., 60., 61., 62., 63.]]])


On this 8x8 grid, we want to consider a split into 16 2x2 subdomains:

In [6]:
split = split_grids(test, kernel_width=4, kernel_height=4)
print(f"split shape: {split.shape}")
# we can visualize the first convolutional block we receive this way
print(split[0,0].reshape(4,4))

split shape: torch.Size([1, 49, 16])
tensor([[63., 56., 57., 58.],
        [ 7.,  0.,  1.,  2.],
        [15.,  8.,  9., 10.],
        [23., 16., 17., 18.]])


In the cell above, we see that when we reshape the vector into a 2D block, we indeed get the top-left 2x2 subgrid with its Moore neighborhood.

The 3rd convolutional block on the other hand corresponds to the next subgrid:

In [7]:
print(split[0,2].reshape(4,4))

tensor([[57., 58., 59., 60.],
        [ 1.,  2.,  3.,  4.],
        [ 9., 10., 11., 12.],
        [17., 18., 19., 20.]])


We can create 4 mutually exclusive sets of subgrids that have at least a spacing of two columns/rows between each other:

In [8]:
subset1 = split[0, [0, 4, 28, 32]]
print(subset1.reshape(4,4,4))

tensor([[[63., 56., 57., 58.],
         [ 7.,  0.,  1.,  2.],
         [15.,  8.,  9., 10.],
         [23., 16., 17., 18.]],

        [[59., 60., 61., 62.],
         [ 3.,  4.,  5.,  6.],
         [11., 12., 13., 14.],
         [19., 20., 21., 22.]],

        [[31., 24., 25., 26.],
         [39., 32., 33., 34.],
         [47., 40., 41., 42.],
         [55., 48., 49., 50.]],

        [[27., 28., 29., 30.],
         [35., 36., 37., 38.],
         [43., 44., 45., 46.],
         [51., 52., 53., 54.]]])


In [9]:
subset2 = split[0, [2, 6, 30, 34]]
print(subset2.reshape(4,4,4))

tensor([[[57., 58., 59., 60.],
         [ 1.,  2.,  3.,  4.],
         [ 9., 10., 11., 12.],
         [17., 18., 19., 20.]],

        [[61., 62., 63., 56.],
         [ 5.,  6.,  7.,  0.],
         [13., 14., 15.,  8.],
         [21., 22., 23., 16.]],

        [[25., 26., 27., 28.],
         [33., 34., 35., 36.],
         [41., 42., 43., 44.],
         [49., 50., 51., 52.]],

        [[29., 30., 31., 24.],
         [37., 38., 39., 32.],
         [45., 46., 47., 40.],
         [53., 54., 55., 48.]]])


In [10]:
subset3 = split[0, [16, 20, 44, 48]]
print(subset3.reshape(4,4,4))

tensor([[[ 9., 10., 11., 12.],
         [17., 18., 19., 20.],
         [25., 26., 27., 28.],
         [33., 34., 35., 36.]],

        [[13., 14., 15.,  8.],
         [21., 22., 23., 16.],
         [29., 30., 31., 24.],
         [37., 38., 39., 32.]],

        [[41., 42., 43., 44.],
         [49., 50., 51., 52.],
         [57., 58., 59., 60.],
         [ 1.,  2.,  3.,  4.]],

        [[45., 46., 47., 40.],
         [53., 54., 55., 48.],
         [61., 62., 63., 56.],
         [ 5.,  6.,  7.,  0.]]])


In [17]:
subset4 = split[0, [14, 18, 42, 46]]
print(subset4.reshape(4,4,4))

tensor([[[15.,  8.,  9., 10.],
         [23., 16., 17., 18.],
         [31., 24., 25., 26.],
         [39., 32., 33., 34.]],

        [[11., 12., 13., 14.],
         [19., 20., 21., 22.],
         [27., 28., 29., 30.],
         [35., 36., 37., 38.]],

        [[47., 40., 41., 42.],
         [55., 48., 49., 50.],
         [63., 56., 57., 58.],
         [ 7.,  0.,  1.,  2.]],

        [[43., 44., 45., 46.],
         [51., 52., 53., 54.],
         [59., 60., 61., 62.],
         [ 3.,  4.,  5.,  6.]]])


In [23]:
grid_subsets = t.cat((
    subset1.reshape(4,4,4).unsqueeze(0),
    subset2.reshape(4,4,4).unsqueeze(0),
    subset3.reshape(4,4,4).unsqueeze(0),
    subset4.reshape(4,4,4).unsqueeze(0)
))
grid_subsets.shape

torch.Size([4, 4, 4, 4])

## Simple update probability function

In [24]:
def p_update(subdomains, cur_vol, target_vol, temperature, src_coords, tgt_coords): 
    src_x, src_y = src_coords
    tgt_x, tgt_y = tgt_coords
    vol_change = (-1 * subdomains[:, tgt_x, tgt_y]) + subdomains[:, src_x, src_y]
    adjusted_vol = cur_vol + vol_change
    if adjusted_vol > 2:
        return 0
    elif adjusted_vol <= 0:
        return 0
    return t.exp((target_vol - cur_vol)**2/temperature)

## Monte Carlo step scheme

In [69]:
import random
def MCS(batch, checkerboard_batch: t.Tensor, target_vol, temperature):
    _, tiles_per_batch, subgrid_width, subgrid_height = checkerboard_batch.shape
    print(f"tiles per batch: {tiles_per_batch}")
    vol_kernel = t.tensor([[[
        [0., 0., 0.],
        [0., 1., 0.],
        [0., 0., 0.]
    ]]])
    padded_batch = periodic_padding(batch).unsqueeze(1)
    cur_vol = t.sum(t.nn.functional.conv2d(padded_batch, vol_kernel))
    print(f"current volume: {cur_vol}")
    # iterate over the sets of spaced out sub-grids
    for subdomains in checkerboard_batch:
        src_x = t.randint_like(t.zeros(tiles_per_batch,), low=1, high=int(subgrid_width-2+1))
        src_x = src_x.type(t.long)
        src_y = t.randint_like(src_x, low=1, high=int(subgrid_height-2+1))
        # For each random sample in src, we sample a random value from [-1, 0, 1]
        # and add it on to the src_idx
        print(t.tensor(random.choices([-1, 1], k=tiles_per_batch)))
        tgt_x = src_x + t.tensor(random.choices([-1, 1], k=tiles_per_batch))
        tgt_y = src_y + t.tensor(random.choices([-1, 1], k=tiles_per_batch))
        print(f"src_x: {src_x}")
        print(f"tgt_x: {tgt_x}")
        print(f"src_y: {src_y}")
        print(f"tgt_y: {tgt_y}")
        
        update_probabilities = p_update(
            subdomains,
            cur_vol,
            target_vol,
            temperature,
            (src_x, src_y),
            (tgt_x, tgt_y)
        )

In [70]:
test = t.zeros(1,8,8)
subdomains = split_grids(test, kernel_width=4, kernel_height=4)
tile_set1 = subdomains[0, [0, 4, 28, 32]]
tile_set2 = subdomains[0, [2, 6, 30, 34]]
tile_set3 = subdomains[0, [16, 20, 44, 48]]
tile_set4 = subdomains[0, [14, 18, 42, 46]]
checkerboard_batch = t.cat((
    tile_set1.reshape(4,4,4).unsqueeze(0),
    tile_set2.reshape(4,4,4).unsqueeze(0),
    tile_set3.reshape(4,4,4).unsqueeze(0),
    tile_set4.reshape(4,4,4).unsqueeze(0)
))
target_vol = 1.
temperature = 27.0

MCS(test, checkerboard_batch, target_vol, temperature)

tiles per batch: 4
current volume: 0.0
tensor([-1,  1, -1,  1])
src_x: tensor([2, 1, 2, 2])
tgt_x: tensor([1, 2, 1, 3])
src_y: tensor([1, 1, 1, 1])
tgt_y: tensor([0, 2, 0, 0])


RuntimeError: Boolean value of Tensor with more than one value is ambiguous